In [22]:
#importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [3]:
#loading dataset
df=pd.read_csv('Netflix_titles.csv')

In [4]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,25-Sep-21,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,24-Sep-21,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,24-Sep-21,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,24-Sep-21,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


## Data cleaning and Modelling

In [5]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [9]:
df['director'].fillna(value='Unknown director',inplace=True)

In [10]:
df['cast'].fillna(value='Unknown cast',inplace=True)

In [12]:
# Now required columns are directors and cast and title only
# Making new column which is combination of director and cast

df['combine']=df['title'] + ' ' + df['director'] + ' '+ df['cast']

In [15]:
# Have to convert into numerical value
t=TfidfVectorizer()
t1=t.fit_transform(df['combine'])

In [16]:
print(t1)

  (0, 6524)	0.18691797765923202
  (0, 38105)	0.11328296157403002
  (0, 19434)	0.40961014398957324
  (0, 8925)	0.38350726748216557
  (0, 16825)	0.3354673803728432
  (0, 17812)	0.5920114988226451
  (0, 9549)	0.4175576223008638
  (1, 24512)	0.1608008383326661
  (1, 27976)	0.08590929285803266
  (1, 24191)	0.09569903491873553
  (1, 33200)	0.16805743889144198
  (1, 39733)	0.09164275294260502
  (1, 10196)	0.14563676624808325
  (1, 32582)	0.13754759794253998
  (1, 32005)	0.15565219829938737
  (1, 23155)	0.16805743889144198
  (1, 23717)	0.16805743889144198
  (1, 14446)	0.16805743889144198
  (1, 26777)	0.16805743889144198
  (1, 25847)	0.1608008383326661
  (1, 18126)	0.12147715603100517
  (1, 21952)	0.16805743889144198
  (1, 32837)	0.1608008383326661
  (1, 12126)	0.1608008383326661
  (1, 14110)	0.16805743889144198
  :	:
  (8805, 31443)	0.15859058009181645
  (8805, 37010)	0.16841484499124215
  (8805, 19084)	0.14391108897571278
  (8805, 24021)	0.11263740061581694
  (8805, 18694)	0.15936867740828117

In [19]:
# Need to create cosine simillarity of each movies
cs=cosine_similarity(t1)

In [21]:
csdf=pd.DataFrame(cs)

# Movie Search Engine

In [71]:
# Getting input
inp=input('Enter the movie title')

d1=difflib.get_close_matches(inp,df['title'])[0]
u=df[df['title']==d1].index[0]
df[df['title']==d1]

Enter the movie titlejeans


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,combine
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,21-Sep-21,1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...,"Jeans S. Shankar Prashanth, Aishwarya Rai Bach..."


# Recommended Movies

In [72]:
recom_movies=csdf.iloc[u].sort_values(ascending=False).iloc[1:11]
recom_movies_df=pd.DataFrame(recom_movies)
recom_movies_df=recom_movies_df.reset_index()
recom_movies_df.columns=['index','simillarity_score']
for i in recom_movies_df['index']:
    print(df[df.index==i]['title'].values)

['Kyun! Ho Gaya Na']
['Guru']
['Shabd']
['Maniyarayile Ashokan']
['Taal']
['Aaviri']
['Jodhaa Akbar']
['Guzaarish']
['Action Replayy']
['Thaandavam']


In [73]:
df[df['country']=='India']['director']

4              Unknown director
24                   S. Shankar
39             Unknown director
50             Unknown director
66             Unknown director
                 ...           
8773               Kedar Shinde
8775           Unknown director
8798    Chandra Prakash Dwivedi
8799             Avadhoot Gupte
8806                Mozez Singh
Name: director, Length: 972, dtype: object